In [ ]:
from PIL import Image
import numpy as np
import os
from tensorflow.keras.utils import to_categorical

IMG_WIDTH = 128
IMG_HEIGHT = 128

def load_classification_data(directory):
    images = []
    labels = []
    class_names = sorted([d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d)) and d != 'annotations'])
    for class_index, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(class_dir, filename)
                img = Image.open(img_path).resize((IMG_WIDTH, IMG_HEIGHT))
                img_array = np.array(img)
                if img_array.shape == (IMG_WIDTH, IMG_HEIGHT, 3):
                    images.append(img_array / 255.0)
                    labels.append(class_index)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names

train_images, train_labels, class_names = load_classification_data('/content/drive/MyDrive/DIP Data Upload/train')

# Verify shapes and data types of train_images and train_labels
print(f"Shape of train_images: {train_images.shape}")
print(f"Data type of train_images: {train_images.dtype}")
print(f"Shape of train_labels: {train_labels.shape}")
print(f"Data type of train_labels: {train_labels.dtype}")
print(f"Unique labels in train_labels: {np.unique(train_labels)}")

def load_segmentation_data(directory):
    images = []
    masks = []
    for class_name in ['gun', 'knife', 'shuriken']:
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(class_dir, filename)
                img = Image.open(img_path).resize((IMG_WIDTH, IMG_HEIGHT))
                img_array = np.array(img)
                if img_array.shape == (IMG_WIDTH, IMG_HEIGHT, 3):
                    images.append(img_array / 255.0)

        mask_class_dir = os.path.join(directory, 'annotations', class_name)
        for filename in os.listdir(mask_class_dir):
            if filename.endswith('.png'):
                mask_path = os.path.join(mask_class_dir, filename)
                mask = Image.open(mask_path).resize((IMG_WIDTH, IMG_HEIGHT)).convert('L')  # Convert to grayscale
                mask_array = np.array(mask) / 255.0
                if mask_array.shape == (IMG_WIDTH, IMG_HEIGHT):
                    masks.append(mask_array)

    # Ensure all masks have the same shape
    mask_shapes = [mask.shape for mask in masks]
    unique_shapes = set(mask_shapes)
    if len(unique_shapes) > 1:
        raise ValueError("Masks have inconsistent shapes.")

    return np.array(images), np.expand_dims(np.array(masks), axis=-1)  # Add channel dimension for masks

train_images_seg, train_masks = load_segmentation_data('/content/drive/MyDrive/DIP Data Upload/train')

print(f"Shape of train_images_seg: {train_images_seg.shape}")
print(f"Data type of train_images_seg: {train_images_seg.dtype}")
print(f"Shape of train_masks: {train_masks.shape}")
print(f"Data type of train_masks: {train_masks.dtype}")


Shape of train_images: (1886, 128, 128, 3)
Data type of train_images: float64
Shape of train_labels: (1886,)
Data type of train_labels: int64
Unique labels in train_labels: [0 1 2 3]
Shape of train_images_seg: (1775, 128, 128, 3)
Data type of train_images_seg: float64
Shape of train_masks: (1834, 128, 128, 1)
Data type of train_masks: float64


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Build the classification model
classification_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
classification_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
classification_model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
48/48 [==============================] - 44s 897ms/step - loss: 0.7951 - accuracy: 0.5643 - val_loss: 4.9121 - val_accuracy: 0.0582
Epoch 2/10
48/48 [==============================] - 40s 831ms/step - loss: 0.5965 - accuracy: 0.6393 - val_loss: 5.4450 - val_accuracy: 0.2698
Epoch 3/10
48/48 [==============================] - 48s 1s/step - loss: 0.5546 - accuracy: 0.6711 - val_loss: 6.2988 - val_accuracy: 0.2249
Epoch 4/10
48/48 [==============================] - 41s 848ms/step - loss: 0.5367 - accuracy: 0.6810 - val_loss: 6.1782 - val_accuracy: 0.1005
Epoch 5/10
48/48 [==============================] - 43s 886ms/step - loss: 0.5196 - accuracy: 0.7082 - val_loss: 8.0875 - val_accuracy: 0.1429
Epoch 6/10
48/48 [==============================] - 41s 862ms/step - loss: 0.4987 - accuracy: 0.7241 - val_loss: 7.3138 - val_accuracy: 0.1323
Epoch 7/10
48/48 [==============================] - 41s 851ms/step - loss: 0.4795 - accuracy: 0.7381 - val_loss: 8.7355 - val_accuracy: 0.2381
Ep

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score

def load_data(directory):
    images = []
    labels = []
    class_names = sorted([d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d)) and d != 'annotations'])
    for class_index, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(class_dir, filename)
                img = Image.open(img_path).resize((IMG_WIDTH, IMG_HEIGHT))
                img_array = np.array(img)
                if img_array.shape == (IMG_WIDTH, IMG_HEIGHT, 3):
                    images.append(img_array / 255.0)
                    labels.append(class_index)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names


# Assuming val_images and val_labels are your validation data
val_images, val_labels, classnames1 = load_data('/content/drive/MyDrive/DIP Data Upload/test')  # Adjust path as needed

# Make predictions on the validation set
val_predictions = classification_model.predict(val_images)
val_pred_labels = np.argmax(val_predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_pred_labels)
print(f"Accuracy: {accuracy}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(val_labels, val_pred_labels)
print(f"Confusion Matrix:\n{conf_matrix}")

16/16 [==============================] - 5s 319ms/step
Accuracy: 0.4178498985801217
Confusion Matrix:
[[113 134   0   0]
 [ 74  93   0   0]
 [ 42  13   0   0]
 [ 22   2   0   0]]


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

def dice_coefficient(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def unet_model(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', dice_coefficient])

    return model

segmentation_model = unet_model()
segmentation_model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 128, 128, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 conv2d_3 (Conv2D)           (None, 128, 128, 64)         36928     ['conv2d_2[0][0]']            
                                                                                                  
 max_pooling2d_2 (MaxPoolin  (None, 64, 64, 64)           0         ['conv2d_3[0][0]']            
 g2D)                                                                                         

In [ ]:
segmentation_model.fit(train_images_seg, train_masks, epochs=1, batch_size=8, validation_split=0.2)


178/178 [==============================] - 3325s 19s/step - loss: 0.0028 - accuracy: 0.9423 - dice_coefficient: 0.0132 - val_loss: 0.0062 - val_accuracy: 0.9039 - val_dice_coefficient: 0.0077


In [ ]:
import cv2

def calculate_dice_coefficient(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (np.sum(y_true_f) + np.sum(y_pred_f) + 1)

def inference_on_test_data(test_directory, classification_model, segmentation_model):
    class_names = ['gun', 'knife', 'shuriken', 'safe']
    output_directory = '/content/drive/MyDrive/DIP Data Upload/output'
    os.makedirs(output_directory, exist_ok=True)

    for class_name in class_names:
        class_dir = os.path.join(test_directory, class_name)
        output_class_dir = os.path.join(output_directory, class_name)
        os.makedirs(output_class_dir, exist_ok=True)

        for filename in os.listdir(class_dir):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(class_dir, filename)
                img = Image.open(img_path).resize((IMG_WIDTH, IMG_HEIGHT))
                img_array = np.array(img) / 255.0

                # Classification
                pred_class = np.argmax(classification_model.predict(np.expand_dims(img_array, axis=0)), axis=1)[0]
                pred_class_name = class_names[pred_class]

                # Segmentation
                if pred_class_name in ['gun', 'knife', 'shuriken']:
                    mask_pred = segmentation_model.predict(np.expand_dims(img_array, axis=0))[0]
                    mask_pred = (mask_pred > 0.5).astype(np.uint8)  # Threshold the prediction
                    mask_pred_resized = cv2.resize(mask_pred, (img.width, img.height), interpolation=cv2.INTER_NEAREST)
                    segmented_image = cv2.bitwise_and(np.array(img), np.array(img), mask=mask_pred_resized.squeeze())

                    # Calculate and print Dice coefficient
                    mask_true_path = os.path.join(test_directory, 'annotations', pred_class_name, filename)
                    if os.path.exists(mask_true_path):
                        mask_true = Image.open(mask_true_path).resize((img.width, img.height)).convert('L')
                        mask_true_array = np.array(mask_true) / 255.0
                        dice_coeff = calculate_dice_coefficient(mask_true_array, mask_pred_resized.squeeze())
                        print(f"Dice coefficient for {filename} ({pred_class_name}): {dice_coeff:.4f}")

                    segmented_image_path = os.path.join(output_class_dir, filename)
                    cv2.imwrite(segmented_image_path, segmented_image)
                else:
                    img.save(os.path.join(output_class_dir, filename))

# Run inference on the test data
test_directory = '/content/drive/MyDrive/DIP Data Upload/test'
inference_on_test_data(test_directory, classification_model, segmentation_model)
